In [1]:
################
# GA parameters:
################
NGEN = 2
MU = 4; NGEN = 3; CXPB = 0.9
USE_CACHED_GA = False
# about 8 models will be made, excluding rheobase search.

################
# Grid search parameters:
# only 5 models, will be made excluding rheobase search
################
npoints = 5
nparams = 1
USE_CACHED_GS = False

# !ulimit -n 2048 
# There is on issue on the Mac with the number of open file handles.  
# I think there is a file leak in LEMS or somewhere else and the command above needs to be run
# in the shell that spawned this notebook before this code can be run to completion.

In [2]:
import pickle
import numpy as np

In [3]:
from neuronunit.optimization.nsga_object import NSGA
from neuronunit.optimization import exhaustive_search as es

Getting Rheobase cached data value for from AIBS dataset 354190013
<class 'neuronunit.tests.passive.InputResistanceTest'> None
Getting cached Input Resistance data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.passive.TimeConstantTest'> None
Getting cached Membrane Time Constant data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.passive.CapacitanceTest'> None
Getting cached Cell Capacitance data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.passive.RestingPotentialTest'> None
Getting cached Resting membrane potential data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.waveform.InjectedCurrentAPWidthTest'> None
Getting cached Spike Half-Width data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.waveform.InjectedCurrentAPAmplitudeTest'> None
Getting cached Spike Amplitude data values from neuroelectro.org
Using cached value.
<class 'neuronunit.tests.wav

In [4]:
if USE_CACHED_GA:
    [invalid_dtc, pop, logbook, fitnesses] = pickle.load(open('ga_dump.p','rb'))
else:
    NSGAO = NSGA()
    NSGAO.setnparams(nparams=nparams)
    invalid_dtc, pop, logbook, fitnesses = NSGAO.main(MU, NGEN)
    with open('ga_dump.p','wb') as f:
       pickle.dump([invalid_dtc, pop, logbook, fitnesses],f)

/Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/__init__.py
Engine IDs:  [<built-in function id>, <built-in function id>, <built-in function id>, <built-in function id>]

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/

In [5]:
if USE_CACHED_GS:
    dtcpopg = pickle.load(open('gr_dump.p','rb'))
else:
    dtcpopg = es.run_grid(npoints,nparams)
    with open('grid_dump.p','wb') as f:
       pickle.dump(dtcpopg,f)

[<neuronunit.optimization.data_transport_container.DataTC object at 0x113f9f080>, <neuronunit.optimization.data_transport_container.DataTC object at 0x1162965c0>, <neuronunit.optimization.data_transport_container.DataTC object at 0x113f9f5c0>, <neuronunit.optimization.data_transport_container.DataTC object at 0x113f9fc18>, <neuronunit.optimization.data_transport_container.DataTC object at 0x113f9fc50>]
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell


/Users/rgerkin/Dropbox/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/rgerkin/Dropbox/miniconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting simulation in NEURON of 500.0ms generated from NeuroML2 model...

Population RS_pop contains 1 instance(s) of component: RS of type: izhikevich2007Cell
Mechanisms already loaded from path: /Users/rgerkin/Dropbox/dev/scidash/neuronunit/neuronunit/models/NeuroML2.  Aborting.

    Starting 

In [6]:
def error(dtc):
    """
    Overall error function for a DTC
    Returns the root-mean-square error over all the tests
    """
    return np.sqrt(np.mean(np.square(list(dtc.scores.values()))))

def sorted_dtcs(dtcpop):
    """
    Returns dtc,error tuples sorted from low to high error
    """
    return sorted([(dtc,error(dtc)) for dtc in dtcpop],key=lambda x:x[1])

minimaga,miniga = sorted_dtcs(invalid_dtc)[0]
minimagr,mini = sorted_dtcs(dtcpopg)[0]
maximagr,maxi = sorted_dtcs(dtcpopg)[-1]

In [7]:
# quantize distance between minimimum error and maximum error.
quantize_distance = list(np.linspace(mini,maxi,10))

# check that the nsga error is in the bottom 1/5th of the entire error range.
print('Report: ')
print("Success" if bool(miniga < quantize_distance[2]) else "Failure")
print("The nsga error %f is in the bottom 1/5th of the entire error range" % miniga)
print("Minimum = %f; 20th percentile = %f; Maximum = %f" % (mini,quantize_distance[2],maxi))

Report: 
Success
The nsga error 0.388812 is in the bottom 1/5th of the entire error range
Minimum = 0.388739; 20th percentile = 0.393128; Maximum = 0.408490


In [8]:
# This function reports on the deltas brute force obtained versus the GA found attributes.
from neuronunit.optimization import model_parameters as modelp
mp = modelp.model_params
for k,v in minimagr.attrs.items():
    #hvgrid = np.linspace(np.min(mp[k]),np.max(mp[k]),10)
    dimension_length = np.max(mp[k]) - np.min(mp[k])
    solution_distance_in_1D = np.abs(float(minimaga.attrs[k]))-np.abs(float(v))
    relative_distance = dimension_length/solution_distance_in_1D
    print('the difference between brute force candidates model parameters and the GA\'s model parameters:')
    print(float(minimaga.attrs[k])-float(v),minimaga.attrs[k],v,k)
    print('the relative distance scaled by the length of the parameter dimension of interest:')
    print(relative_distance)

the difference between brute force candidates model parameters and the GA's model parameters:
-0.051006890386999904 0.893993109613 0.945 a
the relative distance scaled by the length of the parameter dimension of interest:
-18.5269086751


In [9]:
print('the difference between the bf error and the GA\'s error:')
print('grid search:')
from numpy import square, mean, sqrt
rmsg=sqrt(mean(square(list(minimagr.scores.values()))))
print(rmsg)
print('ga:')
rmsga=sqrt(mean(square(list(minimaga.scores.values()))))
print(rmsga)

# Two things to find:
# How close togethor are the model parameters in parameter space (hyper volume), relative to the dimensions of the HV?
# ie get the euclidian distance between the two sets of model parameters.

the difference between the bf error and the GA's error:
grid search:
0.388739448357
ga:
0.388812121512
